# PS 7# 
### Yuqian Gong###

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import svm
%matplotlib inline

### (a) ###

In [2]:
auto = pd.read_csv("Auto.csv", na_values = {"horsepower":["?"]})

In [14]:
auto["const"] = 1
mpg_median = np.median(auto["mpg"])
auto["mpg_condition"] = auto["mpg"] > mpg_median
auto["mpg_high"] = pd.get_dummies(auto["mpg_condition"], drop_first = True)
auto = auto.dropna()
columns = ["mpg_dummy", "mpg_condition"]
auto.drop(columns, inplace = True, axis = 1)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,const,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,1,0


In [15]:
Xvars = auto[["cylinders", "displacement", "horsepower", "weight", "acceleration", "year", "origin", "const"]].values
yvars = auto[["mpg_high"]].values

In [21]:
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(Xvars)
MSE_vec_kf = np.zeros(k)
k_ind = 0
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    MultLogReg = LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg')
    MultLogReg.fit(X_train, y_train)
    y_pred = MultLogReg.predict(X_test)
    MSE_vec_kf[k_ind] = mean_squared_error(y_pred, y_test)
    print(classification_report(y_test, y_pred))
    k_ind += 1

print("The k-fold estimate for the test MSE is", np.mean(MSE_vec_kf))


/Users/Nancygong/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


             precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98



/Users/Nancygong/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:414: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/Nancygong/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


             precision    recall  f1-score   support

          0       0.88      0.94      0.91        47
          1       0.94      0.88      0.91        51

avg / total       0.91      0.91      0.91        98

             precision    recall  f1-score   support

          0       0.89      0.87      0.88        45
          1       0.89      0.91      0.90        53

avg / total       0.89      0.89      0.89        98

             precision    recall  f1-score   support

          0       0.91      0.88      0.90        49
          1       0.88      0.92      0.90        49

avg / total       0.90      0.90      0.90        98

The k-fold estimate for the test MSE is 0.0994897959184


In [22]:
error_rate_low = (1- 0.94 + 1-0.88 + 1 - 0.89 + 1 - 0.91)/4
error_rate_high = (1- 0.87 + 1- 0.94 + 1 - 0.89 + 1- 0.88)/4
print("The average error rate for category 0 is ", error_rate_low )
print("The average error rate for category 1 is ", error_rate_high )

The average error rate for category 0 is  0.095
The average error rate for category 1 is  0.10499999999999995


### (b) ###

In [66]:
Xvars1 = auto[["cylinders", "displacement", "horsepower", "weight", "acceleration", "year", "origin"]].values


random_tree = RandomForestRegressor(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
random_tree.fit(Xvars1, yvars)
random_tree.score(Xvars1, yvars)
y_pred1 = random_tree.oob_prediction_

for i in range(y_pred1.shape[0]):
    if y_pred1[i] >= 0.5:
        y_pred1[i] = 1
    else:
        y_pred1[i] = 0
        
MSE = mean_squared_error(yvars, y_pred1)
print(classification_report(yvars, y_pred1))
print('MSE for random forest approach is ', MSE)

             precision    recall  f1-score   support

          0       0.94      0.91      0.93       196
          1       0.92      0.94      0.93       196

avg / total       0.93      0.93      0.93       392

MSE for random forest approach is  0.0714285714286


/Users/Nancygong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [58]:
error_rate_low_1 = 1 - 0.94
error_rate_high_1 = 1 - 0.92
print("error rate for low category using random forest approach is ", error_rate_low_1)
print("error rate for high category using random forest approach is ", error_rate_high_1)

error rate for low category using random forest approach is  0.06000000000000005
error rate for high category using random forest approach is  0.07999999999999996


### (c) ###

In [63]:
C = 1.0  

k = 4
kf_svm = KFold(n_splits=k, shuffle=True, random_state=15)
kf_svm.get_n_splits(Xvars1)
MSE_vec_kf_svm = np.zeros(k)
k_ind = 0
for train_index, test_index in kf_svm.split(Xvars1):
    X_train2, X_test2 = Xvars1[train_index], Xvars1[test_index]
    y_train2, y_test2 = yvars[train_index], yvars[test_index]
    svm_model = svm.SVC(kernel='rbf', gamma=0.2, C=C)
    svm_model.fit(X_train2, y_train2)
    y_pred2 = svm_model.predict(X_test2)
    MSE_vec_kf_svm[k_ind] = mean_squared_error(y_pred2, y_test2)
    print(classification_report(y_test2, y_pred2))
    k_ind += 1

print("The k-fold estimate for the test MSE using SVM method is", np.mean(MSE_vec_kf_svm))

             precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98

             precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98

             precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98

             precision    recall  f1-score   support

          0       0.53      1.00      0.69        49
          1       1.00      0.10      0.19        49

avg / total       0.76      0.55      0.44        98

The k-fold estimate for the test MSE is 0.507653061224


/Users/Nancygong/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Nancygong/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [64]:
error0_svm = (1 - 1 + 1 - 0.48 + 1 - 0.47 + 1 - 0.53) /4
error1_svm = (1 - 0.45 + 1 - 0 + 1 - 1 + 1 - 1)/4
print("The average error rate for low category using svm method is ", error0_svm)
print("The average error rate for high category using svm method is ", error1b_svm)

The average error rate for low category using svm method is  0.37999999999999995
The average error rate for high category using svm method is  0.38749999999999996


### (d) ###

I think the best predictor of mpg_high is random forest regression. The average error rate for mpg_high is about 0.08 while the average error rate for mpg——high from k-fold logistic regression and svm model are about 0.1 and 0.39 respectively. Besides, the average MSE for random forest models is lowest among the three models, which is only 0.07.